In [1]:
#libreria para el modelo
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

#Para Guardar en Parquet
import os
#import pyarrow as pa
#import pyarrow.parquet as pq


# Carga de Dataframe

In [2]:
df1 = pd.read_csv(r'dataframe\df1.csv')

In [6]:
df1.head(1)

,genres,title,tags,id,developer,Añio,price
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",761140,Kotoshiro,2018,4.99


# Primer Codigo

In [3]:
#Defina un objeto vectorizador TF-IDF. Elimine todas las palabras vacías en inglés, como 'the', 'a' 
tfidf = TfidfVectorizer (stop_words="english")

#Construya la matriz TF-IDF requerida ajustando y transformando Los datos
tfidf_matrix = tfidf.fit_transform(df1['genres'])

#Salida de la forma de tfidf_matrix
#print(tfidf_matrix.shape)
#Matriz de 22496 filas por 31 columnas

#Dado que hemos utilizado el vectorizador TF-IDF, el cálculo del producto escalar nos dará directamente 
#la puntuación de similitud del coseno. Por lo tanto, usaremos linear_kernel () de sklearn en lugar de 
#cosine_similarities() ya que es más rápido.

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

#necesitamos un mecanismo para identificar el índice de una película en nuestro DataFrame de metadatos, dado su título.

#Construya un mapa inverso de índices y títulos de películas.
indices = pd.Series (df1.index, index=df1['id']).drop_duplicates()



## Para Guardar la matriz cosine_sim
 Pero pesa demasiado, mas de 500 MB 

In [5]:
#Se guarda cosien_sim
#(pd.DataFrame(cosine_sim)).to_parquet(os.path.join('dataframe','cosine_sim.parquet'))

#Para abrir el archivo
#cosine_sim = np.array(pd.read_parquet('dataframe\\cosine_sim.parquet'))

In [10]:
#Definamos una función que tome el título de una película como entrada y genere una 
#lista de las 10 películas más similares.

def recomendacion_juego (juego_id, cosine_sim=cosine_sim):

    idx = indices[juego_id]

# Obtenga las puntuaciones de similitud por pares de todas las películas con esa película
    sim_scores = list(enumerate (cosine_sim[idx]))

# Ordene las películas según las puntuaciones de similitud 
    sim_scores = sorted (sim_scores, key=lambda x: x[1], reverse=True)

# Obtén las puntuaciones de las 10 películas más similares 
    sim_scores = sim_scores [1:11]

# Obtenga Los índices de películas

    movie_indices = [i[0] for i in sim_scores]

# Devuelve el top 10 de películas más similares 
    lista = list(df1['id'].iloc[movie_indices])
    lista = lista[0:10]
    

    #Para sacar juegos sin repeticion en los recomendados
    a=None
    lista_juegos=[]
    for i,l in enumerate(lista):
        if not(l in lista_juegos) and (l != id):
           lista_juegos.append(lista[i])
    #del lista_juegos[2]
    
    
    return {
          'Juegos recomendados a partir del juego juego_id' : str(juego_id), 
          'No 1': str(df1[df1['id']==lista_juegos[0]].iloc[0][1]),
          'No 2': str(df1[df1['id']==lista_juegos[1]].iloc[0][1]),
          'No 3': str(df1[df1['id']==lista_juegos[2]].iloc[0][1]),
          'No 4': str(df1[df1['id']==lista_juegos[3]].iloc[0][1]),
          'No 5': str(df1[df1['id']==lista_juegos[4]].iloc[0][1])
          }

In [11]:
recomendacion_juego (670290)



{'Juegos recomendados a partir del juego juego_id': '670290',
 'No 1': 'Snooker-online multiplayer snooker game!',
 'No 2': "Malzbie's Pinball Collection",
 'No 3': 'Zaccaria Pinball - Blackbelt Table',
 'No 4': 'Zaccaria Pinball - Bronze Membership',
 'No 5': 'Zaccaria Pinball - Cine Star Table'}

# Segundo Codigo en prueba

In [ ]:
df1.head()

In [66]:
#filtra las columnas que se usaran para entrenar el modelo
features=['genres', 'developer', 'tags', 'title']
filters=df1[features]



,genres,developer,tags,title
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Kotoshiro,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",Lost Summoner Kitty
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Secret Level SRL,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",Ironbound
2,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Poolians.com,"['Free to Play', 'Simulation', 'Sports', 'Casu...",Real Pool 3D - Poolians
3,"['Action', 'Adventure', 'Casual']",彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222
4,"['Action', 'Adventure', 'Simulation']",Trickjump Games Ltd,"['Action', 'Adventure', 'Simulation', 'FPS', '...",Battle Royale Trainer
...,...,...,...,...
22491,"['Action', 'Adventure', 'Casual', 'Indie']",Bidoniera Games,"['Action', 'Indie', 'Casual', 'Violent', 'Adve...",Kebab it Up!
22492,"['Casual', 'Indie', 'Simulation', 'Strategy']","Nikita ""Ghost_RUS""","['Strategy', 'Indie', 'Casual', 'Simulation']",Colony On Mars
22493,"['Casual', 'Indie', 'Strategy']",Sacada,"['Strategy', 'Indie', 'Casual']",LOGistICAL: South Africa
22494,"['Indie', 'Racing', 'Simulation']",Laush Dmitriy Sergeevich,"['Indie', 'Simulation', 'Racing']",Russian Roads


In [ ]:


#Cleaning the data by making all the words in Lower case. 
def clean_data(x):

     return str.lower(x.replace(" ", ""))

for feature in features:

    filters[feature] = filters[feature].apply(clean_data)

#Ahora podemos crear nuestra "sopa de metadatos", que es una cadena que contiene todos los metadatos 
#que queremos alimentar a nuestro vectorizador.

def create_soup(x):

    return x['genres'] + ' ' + x['developer']+ ' ' +x['tags']+ ' ' + x['title']

filters['soup'] = filters.apply(create_soup, axis=1)    

In [ ]:


#Los siguientes pasos son los mismos que hicimos con nuestro recomendador basado en la descripción de 
#la trama. Una diferencia importante es que usamos CountVectorizer () en lugar de TF-IDF.

# Import CountVectorizer and create the count matrix

from sklearn.feature_extraction. text import CountVectorizer

count = CountVectorizer (stop_words='english') 
count_matrix = count.fit_transform(filters['soup'])

# Compute the Cosine Similarity matrix based on the count_matrix from sklearn.metrics. 
#pairwise import cosine_similarity

cosine_sim2 = cosine_similarity (count_matrix, count_matrix)

In [ ]:
# Restablezca el índice de nuestro DataFrame principal y construya el mapeo inverso como antes filters-filters.reset_index()

indices = pd.Series (filters.index, index=filters['title'])